In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir("../input/human-protein-atlas-image-classification"))
print(os.listdir("../input/protein/saved"))

# Any results you write to the current directory are saved as output.

In [ ]:
# import module we'll need to import our custom module
from shutil import copyfile
import distutils
from distutils import dir_util

distutils.dir_util.copy_tree("../input/proteinproject/project", "../working")

print(os.listdir("../working"))


In [ ]:
%%writefile ../working/config.json
{
    "name": "Protein_Resnet34",
    "n_gpu": 1,
    
    "arch": {
        "type": "Resnet34Model",
        "args": {}
    },
    "data_loader": {
        "type": "ProteinDataLoader",
        "args":{
            "data_dir": "../input/human-protein-atlas-image-classification",
            "batch_size": 50,
            "shuffle": true,
            "validation_split": 0.1,
            "num_workers": 0
        }
    },
    "optimizer": {
        "type": "Adam",
        "args":{
            "lr": 0.0001,
            "weight_decay": 0,
            "amsgrad": true
        }
    },
    "loss": "focal_loss",
    "metrics": [],
    "lr_scheduler": {
        "type": "StepLR",
        "args": {
            "step_size": 5,
            "gamma": 0.1
        }
    },
    "trainer": {
        "epochs": 10,
        "save_dir": "../working/saved/",
        "save_period": 5,
        "verbosity": 2,
        
        "monitor": "min val_loss",
        "early_stop": 2,
        
        "tensorboardX": true,
        "log_dir": "../working/saved/runs"
    }
}

In [ ]:
f = open("../working/config.json", "r")
print(f.read())
f.close()

In [ ]:
import os
import json
import argparse
import torch
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.resnet as module_arch
from trainer import Trainer
from utils import Logger


def get_instance(module, name, config, *args):
    return getattr(module, config[name]['type'])(*args, **config[name]['args'])


def train(config, resume):
    train_logger = Logger()

    # setup data_loader instances
    data_loader = get_instance(module_data, 'data_loader', config)
    valid_data_loader = data_loader.split_validation()

    # build model architecture
    model = get_instance(module_arch, 'arch', config)
    checkpoint = torch.load("../input/protein/saved/Protein_Resnet34_scaled/0325_114954/model_best.pth")
    state_dict = checkpoint['state_dict']
    model.load_state_dict(state_dict)
    print(model)

    # get function handles of loss and metrics
    loss = getattr(module_loss, config['loss'])
    metrics = [getattr(module_metric, met) for met in config['metrics']]

    # build optimizer, learning rate scheduler. delete every lines containing lr_scheduler for disabling scheduler
    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = get_instance(torch.optim, 'optimizer', config, trainable_params)
    lr_scheduler = get_instance(torch.optim.lr_scheduler, 'lr_scheduler', config, optimizer)

    trainer = Trainer(model, loss, metrics, optimizer,
                      resume=resume,
                      config=config,
                      data_loader=data_loader,
                      valid_data_loader=valid_data_loader,
                      lr_scheduler=lr_scheduler,
                      train_logger=train_logger)

    trainer.train()


# Run!
config = json.load(open("../working/config.json"))
path = os.path.join(config['trainer']['save_dir'], config['name'])

train(config, None)

In [ ]:
print("End")